# Program Sets + Mitiq Demo

This notebook demonstrates how to pair Amazon Braket Program Sets with Mitiq zero-noise extrapolation (ZNE).
The goal is to batch together scaled circuits, evaluate them in a single request, and post-process the results.


## Prerequisites

Install dependencies with `uv sync`.


## Workflow Outline

Mitiq supports two main user workflows:

1. A more (mitiq-)coordinated workflow using the `mitiq.xyz.execute_with_xyz` function, which requires a user to define a python function to handle the execution of circuits, and
2. an inspectable workflow where upons specifying the QEM configuration, the circuits are returned and it is expected that the user runs the circuits themselves, and pass the results to a separate mitiq function.

Workflow #1 does not currently work with Program Sets due to "executor" functions needing to have signature looking like `Circuit -> bitstrings | float` or `list[Circuit] -> list[bitstrings] | list[float]`.

For workflow #2, the workflow with Program Sets is the following sequence:

1. Build a reference two-qubit circuit.
2. Generate QEM circuits with Mitiq (in this case Zero-Noise Extrapolation).
3. Package the circuits and observables into a `ProgramSet`.
4. Run the batch on the local Braket simulator and combine results.


In [2]:
from math import pi
from braket.circuits import Circuit
from braket.program_sets import ProgramSet
from braket.circuits.observables import I, Z
import mitiq

In [8]:
# TODO: use a more meaningful circuit and observables
circuit = Circuit().rx(0, pi / 4).ry(1, pi / 8).cnot(1, 0)
observables = [Z(0) @ Z(1), I(0) @ Z(1)]


scale_factors = [1, 3, 5]
qem_circuits = mitiq.zne.construct_circuits(circuit, scale_factors=scale_factors)

In [9]:
program_set = ProgramSet.product(
    circuits=qem_circuits,
    observables=observables,
)  # what other properties of ProgramSet should we showcase?

In [28]:
from braket.devices import LocalSimulator

device = LocalSimulator()  # replace with a noisy device
result = device.run(
    program_set, shots=6 * 100
).result()  # shots must be divisible by 6 (3 scale factors * 2 observables)

In [ ]:
ZZ_expvals = [result[i][0].expectation for i in range(3)]
print("ZZ observable scaled expecation values: ", ZZ_expvals)
print(
    mitiq.zne.combine_results(
        scale_factors,
        ZZ_expvals,
        extrapolation_method=mitiq.zne.inference.LinearFactory.extrapolate,
    )
)


ZZ observable scaled expecation values:  [0.68, 0.66, 0.76]
0.6400000000000001


## Next Steps

- Swap `LocalSimulator` in `run_program_set` for another Braket device once your AWS credentials are configured.
- Experiment with alternative extrapolation factories from Mitiq (for example, `RichardsonFactory`).
- Add more observables to the `ProgramSet.product` call for richer measurement batches.
